In [ ]:
#load in the data
#split data into training and validation data
#use training data to run different evaluation
#evaluate performance on validation data
#different classifiers could be: knearestneighbors (with different neighbors, decisiontreeclasifier)
#analyse results by comparing classifiers on different metrics, inspecting images that are classifies incorrectly etc
#select best classifier (by which rules should we select)
#create python script that takes an image, measures the features, classifies the image
#and gives its probability of being a melanoma (range from 0 to 1)

## Features

* Perimeter
* Area
* Compactness
* Assymetry
* !color

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import Config
from PIL import Image 
import os

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
def load_picture(iid):
    path_mask = '../data/2k/example_segmentation'
    file_mask = path_mask + os.sep + image_id[iid] + '_segmentation.png'
    picture = Image.open(file_mask)
    return picture

In [ ]:
def cut_image(image):
    ''' Assures an image has even number of pixels '''
    width, height = image.size
    image = np.array(image)
    
    if width %2 != 0:
        image = np.delete(image, -1, 1)

    if height %2 != 0:
        image = np.delete(image, -1, 0)

    image = Image.fromarray(image)

    return image

In [ ]:
def assymetry(mask):
    ''' get the assymetry between the left and right part of a given binary mask '''
    width, height = mask.size #mask should be quadratic and therefore have equal dimension
    size = width * height

    #check for uneven number of pixels (should not happen but just as a precaution)
    if width %2 != 0:
        raise TypeError("Uneven number of pixel and cannot be symmetric")
    
    #cut in half and fold
    left = mask.crop((0, 0, (width/2), height)) #left part of picture (left, top, right, bottom)
    right = mask.crop(((width/2), 0, width, height)) #right part of picture
    right = right.transpose(Image.FLIP_LEFT_RIGHT) #flip right part to compare

    #get the binary difference between left an right
    symmetry = np.where(np.array(left) != np.array(right), 1, 0)

    return np.sum(symmetry) / (size/2) #the percentage of assymetry 
    

In [ ]:
def get_assymetry(image):
    ''' get the assymetry for a given mask by folding it in half from multiple angles'''
    return round(np.mean([assymetry(image), assymetry(image.rotate(30, expand= True)),assymetry(image.rotate(60, expand= True)),assymetry(image.rotate(90, expand= True))]),2)

In [ ]:
def compactness(p, A):
    return round(4*np.pi*A / p ** 2, 4)

In [ ]:
def get_features(iid):
    ''' Computes all features for an image id eg. ISIC_19023 '''
    image = load_picture(iid)
    image = cut_image(image)
    p, A = get_perim_area(image)
    assym = assymetry(image)
    comp = compactness(p, A)
    prob = final_classifier(assym, comp, color)
    

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=932aa97b-f6d2-4889-8f04-d6166675cc2f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>